In [1]:
import AdventOfCode

In [2]:
input <- day 22

In [3]:
{-# LANGUAGE OverloadedStrings #-}

import qualified Data.Attoparsec.ByteString.Char8 as A

parseInput = do
    depth <- "depth: " *> A.decimal <* A.endOfLine
    target <- "target: " *> ((,) <$> A.decimal <*> ("," *> A.decimal))
    pure (depth, target)

(depth, target) = parsed parseInput input

In [4]:
import Data.Function.Memoize

geologicIndex :: (Int, Int) -> Int
geologicIndex = memoize $ \c -> case c of
    (0,0) -> 0
    t | t == target -> 0
    (x,0) -> x*16807
    (0,y) -> y*48271
    (x,y) -> erosionLevel (x-1,y) * erosionLevel (x,y-1)
    
erosionLevel = memoize $ \c -> (geologicIndex c + depth) `mod` 20183

riskLevel = memoize $ \c -> erosionLevel c `mod` 3 

In [5]:
import Data.List

sum $ map riskLevel [(x,y) | let (maxX,maxY) = target, x <- [0..maxX], y <- [0..maxY]]

11359

In [6]:
import qualified Data.Set as Set
import qualified Data.Map.Strict as Map

data Equipment = Torch | ClimbingGear | Neither deriving (Eq, Show, Ord)

data CaveState
    = CaveState
    { current :: (Int, Int)
    , equipped :: Equipment
    } deriving (Eq, Show, Ord)

data Measure a = Measure { measure :: Int, value :: a } deriving (Eq, Show, Ord)

distanceFrom (x,y) (a,b) = abs (x-a) + abs (y-b)

changeEquipment :: CaveState -> Int -> (CaveState, Int)
changeEquipment cs@(CaveState curr equip) mins = case riskLevel curr of
    0 -> case equip of
        ClimbingGear -> (CaveState curr Torch, mins+7)
        Torch -> (CaveState curr ClimbingGear, mins+7)
    1 -> case equip of
        ClimbingGear -> (CaveState curr Neither, mins+7)
        Neither -> (CaveState curr ClimbingGear, mins+7)
    2 -> case equip of
        Neither -> (CaveState curr Torch, mins+7)
        Torch -> (CaveState curr Neither, mins+7)

moves :: CaveState -> Int -> [(CaveState, Int)]
moves cs@(CaveState c@(x,y) equip) mins = let
    possible = filter (\(a,b) -> a >= 0 && a < 50 && b >= 0) $ [(x-1,y), (x+1,y), (x,y-1), (x,y+1)]
    in (map (transition cs mins) possible)

transition :: CaveState -> Int -> (Int, Int) -> (CaveState, Int)
transition cs@(CaveState curr equip) mins coord = case (riskLevel coord, equip) of
    (0, c) | c `elem` [ClimbingGear, Torch] -> (CaveState coord equip, mins+1)
    (1, c) | c `elem` [ClimbingGear, Neither] -> (CaveState coord equip, mins+1)
    (2, c) | c `elem` [Neither, Torch] -> (CaveState coord equip, mins+1)
    _ -> let
        (cs', mins') = changeEquipment cs mins
        in transition cs' mins' coord

In [7]:
heuristic :: CaveState -> Int
heuristic (CaveState coord tool) = distanceFrom target coord + (if tool == Torch then 0 else 7)

prioritise :: (CaveState, Int) -> Measure (CaveState, Int)
prioritise c@(cs, cost) = Measure (cost + heuristic cs) c

search :: Map.Map CaveState Int -> Set.Set (Measure (CaveState, Int)) -> (CaveState, Int)
search seen frontier = case value (Set.findMin frontier) of
    final@((CaveState t Torch), c) | t == target -> final
    (cs, c)
        | Map.member cs seen && c >= (seen Map.! cs) -> search seen (Set.deleteMin frontier)
        | otherwise -> let
            seen' = Map.insert cs c seen
            frontier' = Set.deleteMin frontier
            neighbours = moves cs c
            in search seen' (Set.union frontier' (Set.fromList $ map prioritise neighbours))

In [8]:
start = (CaveState (0,0) Torch, 0)

search Map.empty $ Set.singleton (prioritise start)

(CaveState {current = (15,700), equipped = Torch},976)